In [1]:
intcode = [109,424,203,1,21102,11,1,0,1106,0,282,21101,18,0,0,1106,0,259,2101,0,1,221,203,1,21102,31,1,0,1106,0,282,21102,1,38,0,1105,1,259,20102,1,23,2,22101,0,1,3,21101,0,1,1,21101,0,57,0,1106,0,303,1202,1,1,222,21001,221,0,3,20102,1,221,2,21102,259,1,1,21101,80,0,0,1105,1,225,21102,1,149,2,21101,0,91,0,1105,1,303,1202,1,1,223,21002,222,1,4,21102,259,1,3,21102,225,1,2,21102,225,1,1,21101,118,0,0,1105,1,225,20102,1,222,3,21101,0,127,2,21102,133,1,0,1105,1,303,21202,1,-1,1,22001,223,1,1,21102,1,148,0,1106,0,259,1201,1,0,223,21001,221,0,4,21002,222,1,3,21102,14,1,2,1001,132,-2,224,1002,224,2,224,1001,224,3,224,1002,132,-1,132,1,224,132,224,21001,224,1,1,21101,195,0,0,106,0,108,20207,1,223,2,20102,1,23,1,21101,0,-1,3,21102,214,1,0,1106,0,303,22101,1,1,1,204,1,99,0,0,0,0,109,5,1202,-4,1,249,22102,1,-3,1,21201,-2,0,2,21201,-1,0,3,21102,1,250,0,1105,1,225,22102,1,1,-4,109,-5,2106,0,0,109,3,22107,0,-2,-1,21202,-1,2,-1,21201,-1,-1,-1,22202,-1,-2,-2,109,-3,2105,1,0,109,3,21207,-2,0,-1,1206,-1,294,104,0,99,21202,-2,1,-2,109,-3,2106,0,0,109,5,22207,-3,-4,-1,1206,-1,346,22201,-4,-3,-4,21202,-3,-1,-1,22201,-4,-1,2,21202,2,-1,-1,22201,-4,-1,1,22101,0,-2,3,21101,343,0,0,1106,0,303,1106,0,415,22207,-2,-3,-1,1206,-1,387,22201,-3,-2,-3,21202,-2,-1,-1,22201,-3,-1,3,21202,3,-1,-1,22201,-3,-1,2,22101,0,-4,1,21102,1,384,0,1106,0,303,1105,1,415,21202,-4,-1,-4,22201,-4,-3,-4,22202,-3,-2,-2,22202,-2,-4,-4,22202,-3,-2,-3,21202,-4,-1,-2,22201,-3,-2,1,22102,1,1,-4,109,-5,2106,0,0]

In [2]:
from operator import add, mul

def compute_intcode(intcode, inputs=None):
    
    def read():
        mode3, mode2, mode1, code1, code2 = map(int, f'{intcode[pointer]:05}')
        return code1 * 10 + code2, mode1, mode2, mode3
    
    def getv(mode, shift):
        if mode == 1:
            value = intcode.get(pointer + shift, 0)
        elif mode == 2:
            value = intcode.get(intcode.get(pointer + shift, 0) + relative_base, 0)
        else:
            value = intcode.get(intcode.get(pointer + shift, 0), 0)
        return value
    
    def setv(mode, value, shift=3):
        if mode == 1:
            intcode[pointer + shift] = value
        elif mode == 2:
            intcode[intcode[pointer + shift] + relative_base] = value
        else:
            intcode[intcode[pointer + shift]] = value
    
    coords = tuple(inputs)
    inputs = list(inputs or [])
    
    intcode = {i: val for i, val in enumerate(intcode)}
    pointer, relative_base = 0, 0
    opcode, mode1, mode2, mode3 = read()
    while opcode != 99:
        if opcode == 1:
            setv(mode3, add(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 2:
            setv(mode3, mul(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 3:
            value = inputs.pop(0)
            setv(mode1, value, shift=1)
            pointer += 2
        elif opcode == 4:
            value = getv(mode1, 1)
            return coords, value
            pointer += 2
        elif opcode == 5:
            pointer = getv(mode2, 2) if getv(mode1, 1) else pointer + 3
        elif opcode == 6:
            pointer = getv(mode2, 2) if not getv(mode1, 1) else pointer + 3
        elif opcode == 7:
            setv(mode3, 1) if getv(mode1, 1) < getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        elif opcode == 8:
            setv(mode3, 1) if getv(mode1, 1) == getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        elif opcode == 9:
            relative_base += getv(mode1, 1)
            pointer += 2
        opcode, mode1, mode2, mode3 = read()
    return 0

In [3]:
from datetime import datetime
from multiprocessing import cpu_count
from multiprocessing.pool import Pool

def func(inputs):
    return compute_intcode(intcode, inputs)

In [4]:
size = 50
start = datetime.now()
with Pool(cpu_count()) as pool:
    data = pool.map(func, ((x, y) for y in range(size) for x in range(size)))
result = sum(value for key, value in data)
print("Time elapsed:", (datetime.now() - start).total_seconds())
result

Time elapsed: 0.729789


112

In [5]:
shift, size, check = 1800, 2100, 99
start = datetime.now()
with Pool(cpu_count()) as pool:
    data = pool.map(func, ((x, y) for y in range(shift, size) for x in range(shift, size)))
data = {k: v for k, v in data}
print("Time elapsed:", (datetime.now() - start).total_seconds())

start = datetime.now()
result = None
for y in range(shift, size - check):
    for x in range(shift, size - check):
        if not data.get((x, y)):
            continue
        value = True
        for (sx, sy) in ((0, 0), (0, check), (check, 0), (check, check)):
            value &= bool(data.get((x + sx, y + sy), 0))
        if value:
            result = x * 10000 + y
            break
    if result:
        break
print("Time elapsed:", (datetime.now() - start).total_seconds())
(x, y), result

Time elapsed: 19.525673
Time elapsed: 0.031894


((1826, 1982), 18261982)